In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout
import csv
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


In [2]:
model_data = pd.read_csv('model_data.csv')

In [3]:
model_data.columns

Index(['id', 'name', 'close_date', 'stage', 'address', 'lat', 'lng',
       'ts_response', 'mobility_score', 'carshare', 'bikeshare', 'ridehailing',
       'masstransit', 'lat_lng', 'closest_ts', 'within_one_tenth',
       'within_one_half', 'within_one', 'within_five'],
      dtype='object')

In [4]:
#feature engineering
#engineer a number out of stage
model_data.stage.value_counts()

Lost                 926
Closed Won           295
Deal Signed           97
Contract Sent         75
Presentation          65
Invoice Sent          64
Proposal Sent         62
Qualified Lead        60
Research/Donation     27
Contract Expired       8
Contract Review        3
Pilot                  1
Name: stage, dtype: int64

In [ ]:
#Closed, Deal Signed, Invoice Sent = 1; otherwise 0

In [8]:
def translate_stage(stage):
    if stage in ['Closed Won', 'Deal Signed', 'Invoice Sent']:
        return(1)
    else:
        return (0)

In [9]:
model_data['y'] = model_data['stage'].apply(translate_stage)

In [92]:
#feature engineering
y = model_data['y']
X = model_data[['lat',
                'lng',
                'mobility_score',
                'carshare',
                'bikeshare',
                'ridehailing',
                'masstransit',
                'closest_ts',
                'within_one_tenth',
                'within_one_half',
                'within_one',
                'within_five'
                ]]

In [ ]:
#may need to delete outliers here, or put a max on closest_ts

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=14)

In [94]:
#calculate baseline
print (y.value_counts())
print (1 - (456 / (1227+456)))

0    1227
1     456
Name: y, dtype: int64
0.7290552584670231


In [95]:
#random forest
model = RandomForestClassifier(max_features = 6, max_depth = 20) 
scores = cross_val_score(model, X_train, y_train, cv=3)
print(scores)
print(np.mean(scores))

[0.70483461 0.71501272 0.70408163]
0.7079763202991121


In [83]:
#######GRADIENT BOOSTING model
model = GradientBoostingClassifier(max_features = 6, max_depth = 100) 
scores = cross_val_score(model, X_train, y_train, cv=3)
print(scores)
print(np.mean(scores))

[0.70229008 0.70737913 0.73214286]
0.7139373561129286


In [84]:
####### ADABoost model
model = AdaBoostClassifier(n_estimators=100) 
scores = cross_val_score(model, X_train, y_train, cv=3)
print(scores)
print(np.mean(scores))

[0.72773537 0.73791349 0.7627551 ]
0.7428013190008828


In [102]:
#Create keras Model
#X_train, X_test, y_train, y_test = train_test_split(Xtr, ytr, test_size=0.30, random_state=11)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
ss = StandardScaler()
X_train = ss.fit_transform(X_train)  #the scaler is fit only to the training data
X_test = ss.transform(X_test)

model = Sequential()

input_units = X_train.shape[1] #number of features in training set
hidden_units = input_units   #hidden layer has the same number of nodes as input

#first input layer
model.add(Dense(hidden_units            
                ,input_dim=input_units  
                ,activation='relu'
                #uncomment this to add L2 regularization
                #,kernel_regularizer=regularizers.l2(0.0001) 
               ))


#hidden layer (try with and without)
node_reduction = 0
model.add(Dense(hidden_units - node_reduction          
                ,input_dim=input_units  
                ,activation='tanh'
                #,kernel_regularizer=regularizers.l2(0.0001) 
               ))
#model.add(Dropout(0.8))

#final layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy'
              ,optimizer='adam'
               #added later 
              ,metrics=['binary_accuracy']
             )

(1178, 12) (1178,)
(505, 12) (505,)


In [103]:
#Run Keras model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
               epochs=60, batch_size=None, verbose=1)

Train on 1178 samples, validate on 505 samples
Epoch 1/60
1178/1178 [==============================] - 1s 767us/step - loss: 0.6203 - binary_accuracy: 0.6800 - val_loss: 0.5787 - val_binary_accuracy: 0.7545
Epoch 2/60
1178/1178 [==============================] - 0s 51us/step - loss: 0.5786 - binary_accuracy: 0.7326 - val_loss: 0.5536 - val_binary_accuracy: 0.7564
Epoch 3/60
1178/1178 [==============================] - 0s 54us/step - loss: 0.5631 - binary_accuracy: 0.7317 - val_loss: 0.5428 - val_binary_accuracy: 0.7584
Epoch 4/60
1178/1178 [==============================] - 0s 51us/step - loss: 0.5524 - binary_accuracy: 0.7504 - val_loss: 0.5391 - val_binary_accuracy: 0.7644
Epoch 5/60
1178/1178 [==============================] - 0s 55us/step - loss: 0.5456 - binary_accuracy: 0.7606 - val_loss: 0.5355 - val_binary_accuracy: 0.7624
Epoch 6/60
1178/1178 [==============================] - 0s 51us/step - loss: 0.5402 - binary_accuracy: 0.7657 - val_loss: 0.5341 - val_binary_accuracy: 0.764

Epoch 52/60
1178/1178 [==============================] - 0s 57us/step - loss: 0.5089 - binary_accuracy: 0.7750 - val_loss: 0.5438 - val_binary_accuracy: 0.7604
Epoch 53/60
1178/1178 [==============================] - 0s 52us/step - loss: 0.5085 - binary_accuracy: 0.7750 - val_loss: 0.5434 - val_binary_accuracy: 0.7604
Epoch 54/60
1178/1178 [==============================] - 0s 56us/step - loss: 0.5086 - binary_accuracy: 0.7750 - val_loss: 0.5440 - val_binary_accuracy: 0.7604
Epoch 55/60
1178/1178 [==============================] - 0s 56us/step - loss: 0.5077 - binary_accuracy: 0.7750 - val_loss: 0.5442 - val_binary_accuracy: 0.7604
Epoch 56/60
1178/1178 [==============================] - 0s 55us/step - loss: 0.5071 - binary_accuracy: 0.7750 - val_loss: 0.5447 - val_binary_accuracy: 0.7584
Epoch 57/60
1178/1178 [==============================] - 0s 54us/step - loss: 0.5068 - binary_accuracy: 0.7742 - val_loss: 0.5438 - val_binary_accuracy: 0.7604
Epoch 58/60
1178/1178 [=================